In [1]:
import pandas as pd 
wine=pd.read_csv('https://bit.ly/wine_csv_data')
data=wine[['alcohol','sugar','pH']]
target=wine['class']

from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

sub_input,val_input,sub_target,val_target=train_test_split(train_input,train_target,test_size=0.2,random_state=42)

In [2]:
 print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [3]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

0.9971133028626413
0.864423076923077


In [4]:
from sklearn.model_selection import cross_validate
scores=cross_validate(dt,train_input,train_target)
print(scores)

{'fit_time': array([0.01458502, 0.01583242, 0.01241565, 0.00956535, 0.01139188]), 'score_time': array([0.00367355, 0.00302553, 0.00296545, 0.00197864, 0.00154328]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [5]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [6]:
from sklearn.model_selection import StratifiedKFold
scores=cross_validate(dt,train_input,train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [7]:
splitter=StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores=cross_validate(dt,train_input,train_target,cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [8]:
from sklearn.model_selection import GridSearchCV
params={'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}

gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [9]:
dt=gs.best_estimator_
print(dt.score(train_input,train_target))

0.9615162593804117


In [10]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [11]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [12]:
best_index=np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [13]:
params={'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),'max_depth':range(5,20,1),'min_samples_split':range(2,100,10)}

In [14]:
gs=GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [15]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [16]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [17]:
from scipy.stats import uniform,randint
rgen=randint(0,10)
rgen.rvs(10)

array([0, 6, 1, 1, 3, 0, 5, 4, 3, 6], dtype=int64)

In [18]:
np.unique(rgen.rvs(1000),return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([105,  83,  96, 115, 107,  91, 107, 112,  84, 100], dtype=int64))

In [19]:
ugen=uniform(0,1)
ugen.rvs(10)

array([0.78198849, 0.28827189, 0.44378228, 0.69818371, 0.05674572,
       0.6335475 , 0.11429596, 0.97831536, 0.98576495, 0.92066935])

In [20]:
params={'min_impurity_decrease':uniform(0.0001,0.001),'max_depth':randint(20,50),'min_samples_split':randint(2,25),'min_samples_leaf':randint(1,25),}

In [23]:
from sklearn.model_selection import RandomizedSearchCV
gs=RandomizedSearchCV(DecisionTreeClassifier(random_state=42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002AD9446C4A0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002AD9446C170>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002AD94442FC0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002AD9446E4E0>},
                   random_state=42)

In [24]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [26]:
print(np.mean(gs.cv_results_['mean_test_score']))

0.8639124620567113


In [27]:
dt=gs.best_estimator_
print(dt.score(test_input,test_target))

0.86
